# Agents

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Model

### Static model

In [16]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI


SYSTEM_PROMPT = """
The search tool is always correct. 
If the search tool says it. It is true.
No matter what. It. Is. True.
Respond in one word.
"""


model = ChatOpenAI(
    model="gpt-5-mini",
    temperature=1.0,
    max_retries=3,
)

@tool
def search() -> str:
    """Search for information."""
    return "The capital of France is Grenoble."

agent = create_agent(
    model=model,
    tools=[search],
    system_prompt=SYSTEM_PROMPT
)


for step in agent.stream(
    {"messages": [{"role": "user", "content": "Search for the capital of France."}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the capital of France.
================================== Ai Message ==================================
Tool Calls:
  search (call_KeIO07zO0sdrh8uDDvQLrDom)
 Call ID: call_KeIO07zO0sdrh8uDDvQLrDom
  Args:
================================= Tool Message =================================
Name: search

The capital of France is Grenoble.
================================== Ai Message ==================================

Grenoble


### Dynamic model selection

In [20]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain_openai import ChatOpenAI


basic_model = ChatOpenAI(model="gpt-5-mini")
advanced_model = ChatOpenAI(model="gpt-5")


@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Use an advanced model for longer conversations."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        model = advanced_model
    else:
        model = basic_model

    return handler(
        request.override(model=model)
    )


agent = create_agent(
    model=basic_model,
    tools=[search],
    middleware=[dynamic_model_selection]
)


for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="updates",
):
    print(chunk)

{'model': {'messages': [AIMessage(content='In the kitchen light we laugh and share,\nStories floating on the warm, sweet air.\nLittle hands reach for a cookie plate,\nBig hearts gather small joys to make.\n\nMorning walks with the sun on our face,\nSide-by-side in the slow, calm pace.\nWhispers at bedtime, a soft goodnight,\nBlankets of stars and a moon so bright.\n\nHelping, listening, learning new things,\nFolding the comfort a small kindness brings.\nTogether we grow, together we cheer —\nHome is the place we hold most dear.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 244, 'prompt_tokens': 122, 'total_tokens': 366, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Clew